In [5]:
import os
import subprocess
import multiprocessing 
import threading
from functools import partial
from PIL import Image

In [6]:
def makedir(dirName):
    cmd="mkdir {}"
    os.system(cmd.format( dirName))

In [7]:
thCount=multiprocessing.cpu_count()
thCount

8

In [9]:
def resizeImg(ID, directoryList):
    print(directoryList[ID]+' ')
    #create a new subfolder in new directory
    makedir(os.path.join(longPathToNew, newDirName, folder, '"'+directoryList[ID]+'"'))
    #in the subfolder of old directory
    fullPath=os.path.join(longPathToOld, oldDirName, folder, directoryList[ID])
    fullPathToNew=os.path.join(longPathToNew, newDirName, folder, directoryList[ID])
    elementList=os.listdir(fullPath)
    if('.DS_Store' in elementList):
        elementList.remove('.DS_Store')
    for element in elementList:
        subjectNo=element[:2]
        pathToElement=os.path.join(fullPath,element)
        img=Image.open(pathToElement, 'r')
        if subjectNo == 'S3':
            img=img.crop((0,216,1706,1944))
        else:
            img=img.crop((1706,216,3413,1944))
        img=img.resize((256,256), resample=Image.LANCZOS)
        img.save(os.path.join(fullPathToNew,element),optimize=True, quality=90)

In [13]:
newDirName='NewResized'
oldDirName='BigFrameData'
longPathToOld='/media/billy/External'
longPathToNew='/home/billy/Desktop'
cmd="mkdir {}"
ignoreList=['CLAP','INCISION','INTUBATION-STYLETE','MEDICATION','PLACE CUFF']
# tinyList=['CHEST-TUBE PREP','KING AIRWAY','SPLINTING']
# smallList=['BLOOD-PRESSURE CUFF','COMBAT GAUZE','COMBAT TOURNIQUET','ECG LEADS','INTUBATION','IO LINE','PULSE-OX','VITAL CHECKING','WRAP HEAD WOUND']
# mediumList=['BAGGING','CHEST-TUBE','IM ADMINISTRATION','IV TOURNIQUET','SUTURING','IV LINE']
# frameGap=5
makedir(os.path.join(longPathToNew,newDirName))
#symbol link from the original file to new location after classification
#symbol link example: ln -sf source destination
folderList=['Test']

for folder in folderList:
    print('Performing operation for '+ folder+'\n')
#     if os.path.exists(os.path.join(longPathToNew, newDirName, folder)):
#         print('Folder '+ folder +' already exists, skip\n')
#         continue
    makedir(os.path.join(longPathToNew, newDirName, folder))
    target="{}/{}".format(oldDirName, folder)
    dirList=os.listdir(os.path.join(longPathToOld, target))
    if('.DS_Store' in dirList):
        dirList.remove('.DS_Store')
    for cate in dirList:
        if cate in ignoreList:
            dirList.remove(cate)
    #dirList=['BAGGING']
    thCount=multiprocessing.cpu_count()
    pool = multiprocessing.Pool(int(thCount/2))
    partial_resizeImg=partial(resizeImg, directoryList=dirList)
    N = len(dirList)
    _=pool.map(partial_resizeImg,range(N))
    pool.close()
    pool.join()

Performing operation for Test

BAGGING 
